In [1]:
from ultralytics import YOLO
from ultralytics.utils.metrics import mask_iou
from torch import bitwise_or, zeros, Tensor
import cv2
import numpy as np
import pandas as pd

model = YOLO('runs/segment/train2/weights/best.pt')

In [24]:
def read_and_normalize_mask(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    image = cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX)

    _, mask = cv2.threshold(image, 0.5, 1, cv2.THRESH_BINARY)

    mask = mask.reshape(-1, int(image.shape[1] * image.shape[0]))

    return Tensor(mask)

def get_iou(yolo_model, img_path):
    results = yolo_model(img_path)
    true_mask = read_and_normalize_mask(img_path.replace("images", "masks"))
    
    assert len(results) == 1, "support only for len 1"
    for r in results:
        if isinstance(r.masks, type(None)): return None
        masks = r.masks.data.reshape(-1, 256*256).cpu()
        masks = np.any(np.array(masks), axis=0)
        masks = masks.reshape(-1, 256*256)
        masks = Tensor(masks)
    
        return mask_iou(true_mask, masks)

# get_iou(model, './data/test(region_growth)/images/2012_01_12__22_59_55_84__SDO_AIA_AIA_193.png').tolist()[0][0]

In [21]:
# get an average iou
import os
import numpy as np

image_directory = "./data/test(region_growth)/images/"

iou_values = []
for image_path in os.listdir(image_directory):
    if image_path.endswith(".png"):
        result = get_iou(model, os.path.join(image_directory, image_path))
        if result != None:
            iou = result.tolist()[0][0]
            iou_values.append(iou)

average_iou = np.mean(iou_values)

print("Average IoU:", average_iou)




image 1/1 /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/images/2016_08_13__23_00_05_84__SDO_AIA_AIA_193.png: 256x256 4 coronal holes, 12.0ms
Speed: 16.6ms preprocess, 12.0ms inference, 1.5ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/images/2015_03_11__22_59_42_84__SDO_AIA_AIA_193.png: 256x256 3 coronal holes, 13.9ms
Speed: 0.5ms preprocess, 13.9ms inference, 1.5ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/images/2013_10_17__22_59_42_84__SDO_AIA_AIA_193_r180.png: 256x256 1 coronal hole, 12.2ms
Speed: 0.3ms preprocess, 12.2ms inference, 1.8ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/images/2014_01_26__23_00_06_84__SDO_AIA_AIA_193_r270.png: 256x256 (no detections), 12.2ms
Speed: 1.8ms preprocess, 12.2ms inference, 0.5ms postproces

Average IoU: 0.7006662135058757


In [3]:
val_res = model.val(data='./bakalarka-yolo.yaml', imgsz=256, split="test")

Ultralytics YOLOv8.0.203 🚀 Python-3.8.5 torch-2.1.0+cu121 CUDA:0 (NVIDIA TITAN RTX, 24210MiB)
YOLOv8x-seg summary (fused): 295 layers, 71721619 parameters, 0 gradients, 343.7 GFLOPs
val: Scanning /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/la
val: New cache created: /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        822       3476      0.821       0.45      0.508      0.402      0.821      0.446      0.505      0.322
Speed: 0.1ms preprocess, 4.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/segment/val6


In [4]:
dice = val_res.seg.f1
dice

array([    0.57781])